# Check `GDS` Python stack

This notebook checks all software requirements for the course Geographic Data Science are correctly installed. 

A successful run of the notebook implies no errors returned in any cell *and* every cell beyond the first one returning a printout of `True`. This ensures a correct environment installed.

---

In [ ]:
import bokeh as bk
float(bk.__version__[:1]) >= 1

In [ ]:
import black

In [ ]:
import matplotlib as mpl
float(mpl.__version__[:3]) >= 1.5

In [ ]:
import mplleaflet as mpll

In [ ]:
import seaborn as sns
float(sns.__version__[:3]) >= 0.6

In [ ]:
import datashader as ds
float(ds.__version__[:3]) >= 0.6

In [ ]:
import palettable as pltt
float(pltt.__version__[:3]) >= 3.1

In [ ]:
sns.palplot(pltt.matplotlib.Viridis_10.hex_colors)

---

In [ ]:
import qgrid

In [ ]:
import pandas as pd
float(pd.__version__[:4]) >= 0.18

In [ ]:
import dask
float(dask.__version__[:1]) >= 1

In [ ]:
import dask_ml

In [ ]:
import sklearn
float(sklearn.__version__[:4]) >= 0.20

In [ ]:
import statsmodels.api as sm
float(sm.version.version[:3]) >= 0.9

In [ ]:
import pymc3 as pm
float(pm.__version__[:3]) >= 3.6 

In [ ]:
import xlrd

In [ ]:
import xlsxwriter

In [ ]:
import networkx as nx

In [ ]:
import osmnx

---

In [ ]:
import fiona
float(fiona.__version__[:3]) >= 1.8

In [ ]:
import geopandas as gpd
float(gpd.__version__[:3]) >= 0.4

In [ ]:
import pysal as ps
float(ps.__version__[:1]) >= 2

In [ ]:
import rasterio as rio
float(rio.__version__[:1]) >= 1

In [ ]:
import rasterstats

# Test

In [ ]:
shp = ps.lib.examples.get_path('columbus.shp')
db = gpd.read_file(shp)
db.head()

In [ ]:
db[['AREA', 'PERIMETER']].to_feather('db.feather')
tst = pd.read_feather('db.feather')
! rm db.feather

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f, ax = plt.subplots(1)
db.plot(facecolor='yellow', ax=ax)
ax.set_axis_off()
plt.show()

In [ ]:
db.crs['init'] = 'epsg:26918'

In [ ]:
db_wgs84 = db.to_crs(epsg=4326)
db_wgs84.plot()
plt.show()

In [ ]:
from pysal.viz.splot.mapping import vba_choropleth

f, ax = vba_choropleth(db['INC'], db['HOVAL'], db)

In [ ]:
# Out until geopandas 0.5 comes in
#db.plot(column='INC', scheme='fisher_jenks', cmap=plt.matplotlib.cm.Blues)
#plt.show()

In [ ]:
city = osmnx.gdf_from_place('Berkeley, California')
osmnx.plot_shape(osmnx.project_gdf(city));

In [ ]:
import numpy as np
import contextily as ctx
tl = ctx.tile_providers.ST_TONER_LITE

db = gpd.read_file(ps.lib.examples.get_path('us48.shp'))
db.crs = {'init': 'epsg:4326'}
w, s, e, n = db.to_crs(epsg=3857).total_bounds
dbp = db.to_crs(epsg=3857)
# Download raster
_ = ctx.bounds2raster(w, s, e, n, 'us.tif', url=tl)
# Load up and plot
source = rio.open('us.tif', 'r')
red = source.read(1)
green = source.read(2)
blue = source.read(3)
pix = np.dstack((red, green, blue))
bounds = (source.bounds.left, source.bounds.right, \
          source.bounds.bottom, source.bounds.top)
f = plt.figure(figsize=(6, 6))
ax = plt.imshow(pix, extent=bounds)

! rm us.tif

In [ ]:
ax = db.to_crs(epsg=3857).plot()
ctx.add_basemap(ax)

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl

m = Map(center=(42.6824, 365.581), zoom=5)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-11-11")
left_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisAquaBands721CR, "2017-11-11")

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m